<a href="https://colab.research.google.com/github/SelinaZheng-LX/SelinaZheng-LX/blob/main/F23_HC7%268_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HC7 Data Exploration an Cleaning

Below you will find a first exploration of the [`NYC_Clean_Heat_Dataset__Historical_.csv`](https://data.cityofnewyork.us/City-Government/NYC-Clean-Heat-Dataset-Historical-/8isn-pgv3) dataset. The goal is for you to learn the tools for data exploration (many of these you learned in our online labs) and then be able to further explore the data and obtain results to report in our final presentation.  

As you know, we must first import the packages we will use:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import gdown

## EXPLORE THE DATA

To begin, we will read the data into a pandas dataframe and find out its shape (dimensions)

In [ ]:
# download the file from our drive
!wget https://huntercsci127.github.io/files/clean_heat_dataset.csv
#list the files in the current directory to confirm the file is there
!ls


In [ ]:
!ls


In [ ]:
# read Affordable_Housing_Production_by_Building.csv into a dataframe
clean_heat = pd.read_csv("clean_heat_dataset.csv")

print("The dimension of the table is: ", clean_heat.shape)

Alternatively, to find out how many observations (rows) in the dataset:

In [ ]:
print("Number of rows: ", len(clean_heat))

To find out the names of the columns, we can print the header of the dataframe (with zero rows)

In [ ]:
print("Column Names:\n",clean_heat.head(0))

Alternatively, to see sample data, we can print the header of the dataframe with the first two rows:

In [ ]:
print("Column Names:\n",clean_heat.head(2))

Oh no! There are too many columns to print, 41 of them! (thats why there is the `...` in between.)  
Let's look at what type of data is stored in each column:

In [ ]:
clean_heat.info()

You can access speicfic columns using exact the same name included in quotes and get the head of the dataframe with the selected columns (as shown below)

In [ ]:
print("Project Name:\n",clean_heat['Boiler Model'].head(5))

We can use `describe()` to get some stats for the numerical data, shown as follows. Some of the output make sense, for example, calculate the average, maximum, or minimum number of 2-BR Units, but some of the output will not be used, for example, the mean of Postcode.

In [ ]:
clean_heat.describe()

The function `describe()` gives the number of values (`count`), average, standard deviation, minimum and maximum values as well as the 25th, 50th and 75th percentiles, **for each numerical column**. As you can see, not all numerical values can be treated equally. For example, it doesn't make much sense to take the average and standard deviation of a postcode or the year of the deadline. It is, however, useful to know what the max and min values for the deadline years are.  
To retrieve and print the stats **for a particular column**:

In [ ]:
num_stats = clean_heat.describe()
print("Statistics for 'Total MMBTU' column:\n",num_stats['Total MMBTU (High)'])

The `describe()` function, as called above, provides statistics for numerical data. To get some information for the categorical data:

In [ ]:
clean_heat.describe( include=['O'])

The `describe()` function with `include=['O']` argument (that is a capital letter O), will provide the number of values (`count`), the number of unique values, the most frequent value (`top`) and the frequency of that value, **for each categorical column**.
You can see that some data that is encoded categorically, like `Bldg Sqft`, shoudl really be numerical, while some numerical data, like `BIN` (Buidling Identification Number), should really be categorical.  
We encoruage you to convert such columns in your data cleaning phase.

As we did above, if you want to extract the stats **for a particular categorical column**:

In [ ]:
cat_stats = clean_heat.describe(include=['O'])
print("Borough:\n", cat_stats['Borough'])

Here we can see that all rows specify the `Borough` (count is the same as the number of rows in the dataset, which is 4789 as we saw earlier). We can also observe that there are 5 boroughs (unique == 5) and that Manhattan (marked by top) has most projects (2597).

Let's look at another column:

In [ ]:
print("Statistics for Burner Model:\n", cat_stats['Burner Model'])

Here we also see that not all rows have an entry for `Burner Model` (count is less than number of rows in the dataset), we see that there are 2624 (marked as unique) different models, the most frequent model is "ICI DEG 42P".

## CLEAN THE DATA  

It is always a good idea to clean your data before you start working. The most common problem you will encouter is missing data. If some rows do not provide a value for a particular feature (column), you may encounter errors when you try to compute statistics or plot the data.   

To check if there are any missing values by suming up the null values for each column:

In [ ]:
print("Number of dataponts with null entry for each column:\n",clean_heat.isnull().sum())

Here we see that there are many columns that have missing values (non-zero).

To find out if there are multiple rows for the same building, we can use `value_counts()` on that column

In [ ]:
clean_heat['BIN'].value_counts()

Indeed, there are multiple entries for the same building, For example, there are 10 occurrences fir BIN 4455390.
So now, **let's clean the data** by removing the null values.  
We can do that by replacing null values for our numerical columns with 0

In [ ]:
columns= ['Borough, Block, Lot #', 'Street Address', 'Postcode', 'Borough', 'Utility', 'Building Manager', 'Owner', 'Owner Address', 'Owner Telephone', 'DEP Boiler Application #', '#6 Deadline', 'Boiler Model', '# of Identical Boilers', 'Boiler Capacity (Gross  BTU)', 'Boiler Installation Date', 'Boiler Age Range', 'Est. Retirement Year', 'Burner Model', 'Primary Fuel', 'Total Gallons (High)', 'Total Gallons (Low)', 'Total MMBTU (High)', 'Total MMBTU (low)', 'Greener Greater Buildings', 'GGB Deadline', 'Building Type', 'Council District', 'Community Board', 'Bldg Sqft', '# of Bldgs', '# of Floors', '# of Res. Units', 'Total Units', 'Year Built', 'Condo?', 'Coop?', 'Latitude', 'Longitude', 'Census Tract', 'BIN', 'BBL', 'NTA']
cat_columns= ['Street Address', 'Borough', 'Utility', 'Building Manager', 'Owner','Owner Address','Owner Telephone', 'DEP Boiler Application #','Boiler Model','Boiler Age Range', 'Burner Model', 'Primary Fuel', 'Building Type', 'Community Board', 'Bldg Sqft', 'Condo?', 'Coop?', 'NTA']
num_columns=['Borough, Block, Lot #','Postcode','#6 Deadline','# of Identical Boilers','Boiler Capacity (Gross  BTU)', 'Boiler Installation Date',  'Est. Retirement Year', 'Total Gallons (Low)', 'Total MMBTU (High)', 'Total MMBTU (low)', 'Greener Greater Buildings', 'GGB Deadline', 'Council District', '# of Bldgs', '# of Floors', '# of Res. Units', 'Total Units', 'Year Built','Latitude', 'Longitude', 'Census Tract', 'BIN', 'BBL']

In [ ]:
#clean_heat.fillna(num_columns:0,inplace=True)
clean_heat[num_columns]=clean_heat[num_columns].fillna(value=0)
# affordable.fillna({'Council District':0},inplace=True)
# affordable.fillna({'Latitude':0, 'Longitude':0},inplace=True)
# affordable.fillna({'Latitude (Internal)':0, 'Longitude (Internal)':0},inplace=True)

And replace null values in our categorical data (string values) with the empty string:

In [ ]:
clean_heat[cat_columns]=clean_heat[cat_columns].fillna(value="")

If you look at the categorical data description, you may find more sensible replacements for some of the columns. We encourage you to do that.

Now we can check to make sure we got rid of all null values:

In [ ]:
print(clean_heat.isnull().sum())

And voila', our data that are of **number** or **string** type are free of null values!!!   

These substitutions may not be the most appropriate for all categorical or numerical data and, as mentioned above, some data encoded as categorical may be more appropriate considered as numerical or vice versa. We encourage you to take care of this in the way that best supports your analysis.


# VISUALIZE THE DATA

### Let's look at the data by borough:  

To find out the names of the boroughs (we kind of know this already...):

In [ ]:
print("Unique Borough Names:\n", clean_heat.Borough.unique())

Let's group the data by borough (`Borough`)

In [ ]:
boro_group = clean_heat.groupby(['Borough'])

Now that the data is grouped by borough in boro_group, let's plot the data to observe the average total gallons per borough:

In [ ]:
boro_group['Total Gallons (High)'].mean().plot.bar()

To save your plot in an image file you can do the following to leave more space at the bottom of the figure to fully visualize the names of the boroughs on the x axis, label the y axis, and then clear the figure so we can make more plots later:

In [ ]:
#create more space for x-axis labels
plt.gcf().subplots_adjust(bottom=0.25)
plt.ylabel('Average total gallons')
#get the current figure
fig = plt.gcf()
#save figure to an image file
fig.savefig('mean_gallons_per_boro.png')

#clear figure to generate next pyplot
plt.clf()


Let's now plot the data to observe Reporting Construction Type by borough:

In [ ]:
#Generate the number of Preservation or New Construction with value_count()
boro_group['Primary Fuel'].value_counts().plot.bar()

Let's say I now want to look at data from Staten Island only (**note I am selecting rows here!!!**)

In [ ]:
#Select only buildings in Staten Island
st = clean_heat[clean_heat['Borough'].isin(['Staten Island'])]
print("Number of entries in Staten Island: ", len(st))

I now have a new dataframe called st with only rows where borough == 'Staten Island", and I can see that there are only 6 entries for Staten Island.

In [ ]:
#Graph data from only Staten Island
st['Primary Fuel'].value_counts().plot.bar()

Now we can see the values and bars for Staten Island!



### YOUR TURN!

Now that you know how to explore the data, clean the data, obtain statistics about the data, visualize the data and select a subset of the data based on the value in a particular column (e.g. neighbourhood_group == 'Staten Island"), think about how you want to explore the data for your analysis:    
  1. As a group, think about an overall data-driven discussion of your borough and how it compares to the others.  
  2. Individually, analyze the data in your borough and compare to the data for other boroughs.
  
As you explore your data, keep in mind your analysis and findings from HC2 and HC3 and see if you can make any connections, or if you find that the data supports those findings.